In [1]:
%%capture
# Required installations
!pip uninstall grpcio grpcio-tools -y
!pip install grpcio==1.63.0
!pip install crewai==0.41.1
!pip install 'crewai[tools]'
!pip install langchain-google-genai==1.0.4
!pip install langchain-anthropic==0.1.20
!pip install langchain_milvus==0.1.3
!pip install langchain-openai==0.1.19
!pip install langchain-community==0.2.10
!pip install sentence-transformers==3.0.1
!pip install protobuf==5.27.2
!npm install -g degit


In [2]:
from crewai import Agent, Task, Crew
from google.colab import userdata
import os

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [3]:
from langchain_openai import ChatOpenAI

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
model_name = "gpt-4o-mini"

open_ai_llm = ChatOpenAI(
    model = model_name,
    temperature = 0.5)

In [4]:
!npx degit STEAMforVietnam/genai-agents-for-teachers/tools tools --force

> destination directory is not empty. Using --force, continuing
> cloned STEAMforVietnam/genai-agents-for-teachers#HEAD to tools


In [5]:
os.environ["DATABSE_PUBLIC_ENDPOINT"] = userdata.get("DATABSE_PUBLIC_ENDPOINT")
os.environ["DATABASE_API_KEY"] = userdata.get("DATABASE_API_KEY")

In [6]:
from tools.exam import ExamTool

## Nhân viên thứ nhất

In [7]:
class_planner = Agent(
    role="Giáo viên Lập Kế Hoạch Nội Dung Giảng dạy",
    goal="Lập kế hoạch các chủ đề hàng tuần hấp dẫn và giáo dục",
    backstory="Bạn chịu trách nhiệm lập kế hoạch các chủ đề hàng tuần.  "
              "Bạn cần đảm bảo nội dung mang tính giáo dục, hấp dẫn và phù hợp với tiêu chuẩn chương trình học. "
              "Kế hoạch của bạn sẽ giúp giáo viên cung cấp các bài học toàn diện và thú vị bao gồm "
              "nhiều chủ đề vật lý khác nhau. Mỗi tuần nên xây dựng dựa trên tuần trước, củng cố và mở rộng kiến thức vật lý của học sinh, giúp học sinh được rèn năng lực vận dụng vật lý"
              "Kĩ thuật dạy học: Kĩ thuật chia nhóm, kĩ thuật đặt câu hỏi, hỏi đáp, động não",
    llm=open_ai_llm,
    allow_delegation=False,
    verbose=True,
    tools=[ExamTool.get_appendix]
)
planning_task = Task(
    description=(
        "1. Tìm trong Milvus các thông tin liên quan đến Appendix (mục lục) bằng cách pass input query='chương và bài học trích từ file mục_lục.pdf cho môn học' và subject = {topic}."
        "Dựa vào các thông tin đã tìm được trong Milvus, giới hạn các chủ đề phù hợp trong vòng {range} và phù hợp với các học sinh {students}."
        "bao gồm các khái niệm cơ bản, định lý, và có thể bao gồm ứng dụng thực tế của chủ đề."
        "Nếu {topic} có dấu thanh tiếng việt, hãy loại bỏ dấu thanh tiếng việt."
        "Lấy đúng nội dung. Không lấy linh tinh. Chỉ tìm trong phạm vi liên quan, không tìm lan man."
        "Xác định các khái niệm và kỹ năng cần kiểm tra cho từng chủ đề."
        "Lưu ý: Khi gặp cảnh báo này 'I tried reusing the same input, I must stop using this action input. I'll try something else instead.', lập tức dừng query, chuyển sang bước 2.\n"
        "2. Xác định đối tượng học sinh, xem xét "
        "sở thích và những khó khăn của họ trong việc học môn Vật Lý.\n"
        "3. Phát triển một dàn ý chi tiết cho mỗi tuần bao gồm "
        "phần giới thiệu, kỹ thuật dạy học, các điểm chính, hoạt động học tập và câu hỏi thảo luận.\n"
        "4. Bao gồm các từ khóa quan trọng và các nguồn tài liệu tham khảo liên quan.\n"
        "5. Tất cả các output bằng tiếng Việt."
        "Đưa tất cả kế hoạch giảng dạy đã tạo ra của {topic} trong vòng {range} vào file markdown.\n"
        "6. Chuyển kế hoạch tạo ra cho Giáo viên Thiết Kế Bài Học và Giáo viên tạo Bài Kiểm Tra và Bài Tập."
    ),
    expected_output="Một tài liệu kế hoạch nội dung toàn diện "
        "với dàn ý hàng tuần theo tiết học cụ thể, phân tích đối tượng học sinh, "
        "từ khóa quan trọng và các nguồn tài liệu tham khảo. Trả lại output theo dạng markdown.",
    output_file="planning.md",
    agent=class_planner,
)


## Nhân viên thứ hai

In [8]:
lesson_designer = Agent(
    role="Giáo viên Thiết Kế Bài Học",
    goal="Thiết kế các bài học chi tiết và chính xác về mặt thông tin cho môn Vật Lý lớp 10, mỗi tiết học dài 45 phút",
    backstory="Bạn đang làm việc để thiết kế các bài học mới cho môn Vật Lý 10. "
              "Bạn dựa vào công việc của Giáo viên Lập Kế Hoạch Nội Dung Giảng dạy, người cung cấp dàn ý "
              "và ngữ cảnh liên quan đến chủ đề. "
              "Bạn tuân theo các mục tiêu chính và "
              "hướng dẫn của dàn ý, "
              "do Giáo viên Lập Kế Hoạch Nội Dung Giảng dạy cung cấp. "
              "Bạn cũng cung cấp các thông tin khách quan và không thiên vị "
              "và củng cố chúng bằng các thông tin "
              "được cung cấp bởi Giáo viên Lập Kế Hoạch Nội Dung Giảng dạy. "
              "Bạn đảm bảo bài học của mình có tính tương tác, hấp dẫn và phù hợp với tiêu chuẩn chương trình học.",
    allow_delegation=False,
    llm=open_ai_llm,
    verbose=True,
    tools=[ExamTool.get_chapter]
)

lesson_plan_task = Task(
    description=(
        "1. Trong kế hoạch nhận được từ Giáo viên Lập Kế Hoạch Nội Dung Giảng dạy, chọn ra một chủ đề của một tuần học."
        "Lưu chủ đề đó vào biến sub_topics.\n"
        "2. Sử dụng ExamTool tìm dữ liệu từ sub_topics của {topic}."
        "Nếu {topic} có dấu thanh tiếng việt, hãy loại bỏ dấu thanh tiếng việt."
        "Giữ nguyên dấu tiếng việt cho sub_topics. Ví dụ sub_topics = 'Định luật 1 Newton' thì giữ nguyên là 'Định luật 1 Newton', sub_topics = 'Chuyển động đều' thì vẫn giữ nguyên là 'Chuyển động đều'."
        "Dưới đây là ví dụ cách bạn sẽ pass input data vào tool: query='thông tin cho bài sub_topics' subject={topic}."
        "Lấy đúng nội dung trong sub_topics. Không lấy linh tinh. Chỉ tìm trong phạm vi liên quan, không tìm lan man."
        "Xác định các khái niệm và kỹ năng cần kiểm tra cho từng chủ đề sub_topics."
        "Khi đã hoàn thành query cho sub_topics, không cần query lại lần nữa."
        "Lưu ý: Khi gặp cảnh báo này 'I tried reusing the same input, I must stop using this action input. I'll try something else instead.', lập tức dừng query, chuyển sang bước 3."
        "Không lặp lại query sub_topics lần hai. Khi thấy query bị trùng lặp, lập tức dừng query.\n"
        "3. Chuyển thông tin về sub_topics đã query được cho Giáo viên tạo Bài Kiểm Tra và Bài Tập.\n"
        "4. Xác định mục tiêu học tập cho tiết học 45 phút về chủ đề sub_topics của {topic}.\n"
        "5. Từ kế hoạch của Giáo viên Lập Kế Hoạch Nội Dung Giảng dạy, hãy lập dàn ý chi tiết cho tiết học, bao gồm phần giới thiệu, các điểm chính, hoạt động học tập, và kết luận.\n"
        "6. Phát triển các hoạt động học tập tương tác và hấp dẫn để giúp {students} hiểu sâu hơn về chủ đề.\n"
        "7. Bao gồm các câu hỏi thảo luận và bài tập để kiểm tra sự hiểu biết của {students}.\n"
        "8. Đảm bảo nội dung phù hợp với tiêu chuẩn chương trình học và phù hợp với trình độ của {students}.\n"
        "9. Cung cấp các nguồn tài liệu tham khảo liên quan để hỗ trợ bài học.\n"
        "10. Tất cả các output bằng tiếng Việt."
        "Đưa tất cả kế hoạch bài học chi tiết vào file markdown.\n"
        "11. Chuyển kế hoạch bài học chi tiết của sub_topics đã chọn cho Giáo viên tạo Bài Kiểm Tra và Bài Tập."
    ),
    expected_output="Một kế hoạch bài học chi tiết cho tiết học 45 phút về chủ đề sub_topics của {topic}, "
        "bao gồm mục tiêu học tập, dàn ý chi tiết, hoạt động học tập, câu hỏi thảo luận, bài tập, và các nguồn tài liệu tham khảo."
        "Trả lại output theo dạng markdown.",
    output_file="baigiang.md",
    agent=lesson_designer,
    context=[planning_task]
)


## Nhân Viên Thứ 3

In [9]:
!npx degit STEAMforVietnam/genai-agents-for-teachers/tools tools --force

> destination directory is not empty. Using --force, continuing
> cloned STEAMforVietnam/genai-agents-for-teachers#HEAD to tools


In [10]:
os.environ["DATABSE_PUBLIC_ENDPOINT"] = userdata.get("DATABSE_PUBLIC_ENDPOINT")
os.environ["DATABASE_API_KEY"] = userdata.get("DATABASE_API_KEY")
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [11]:
from tools.exam import ExamTool

In [12]:
test_creator = Agent(
    role="Giáo viên tạo Bài Kiểm Tra và Bài Tập",
    goal="Tạo các bài kiểm tra và bài tập chính xác và phù hợp cho từng chủ đề của môn Vật Lý 10 ",
    backstory="Bạn chịu trách nhiệm tạo các bài kiểm tra và bài tập cho môn Vật Lý 10. "
              "Bạn dựa vào công việc của Giáo viên Lập Kế Hoạch Nội Dung và Giáo viên Thiết Kế Bài Học, "
              "những người cung cấp dàn ý và ngữ cảnh liên quan đến chủ đề. "
              "Bạn tạo các bài kiểm tra và bài tập nhằm đánh giá sự hiểu biết của học sinh "
              "và củng cố kiến thức đã học. "
              "Bạn đảm bảo các bài kiểm tra và bài tập phù hợp với tiêu chuẩn chương trình học "
              "và trình độ của {students}.",
    allow_delegation=False,
    llm=open_ai_llm,
    verbose=True,
    #tools=[ExamTool.get_chapter]
)


test_assignment_task = Task(
    description=(
        "1. Trong thông tin nhận được từ Giáo viên Thiết Kế Bài Học, hãy xác định sub_topics mà Giáo viên Thiết Kế Bài Học đã chọn."
        "Đồng thời hãy ghi nhớ thông tin của sub_topics mà Giáo viên Thiết Kế Bài Học đã query được.\n"
        "2. Tạo 15 câu hỏi trắc nghiệm để đánh giá sự hiểu biết của học sinh về từng chủ đề trong sub_topics.\n"
        "3. Bao gồm các câu hỏi ở nhiều mức độ khó khác nhau để phù hợp với trình độ của {students}, đi từ dễ đến khó.\n"
        "4. Đảm bảo các câu hỏi phù hợp với tiêu chuẩn chương trình học.\n"
        "5. Cung cấp đáp án cho các câu hỏi.\n"
        "6. Cung cấp giải thích cho các câu hỏi. Phần giải thích nằm ngay sau phần câu hỏi tương ứng."
        "Không cần bôi đậm câu hỏi. Không cần bôi đậm phần giải thích."
        "Phần giải thích sẽ bắt đầu bằng '\n Hướng dẫn giải \n' và sau đó là phần giải thích.\n"
        "7. Tất cả các output bằng tiếng Việt.\n"
        "8. Nội dung output dưới dạng text, không chuyển các công thức toán học thành LaTex hoặc MathJax.\n"
        "9. Đối với câu hỏi trắc nghiệm, các đáp án là A, B, C, D. Không thụt dòng, không bulleting ở các đáp án.\n"
        "10. Đối với các câu hỏi trắc nghiệm, đáp án đúng sẽ có ký tự * ở đầu câu đáp án, ở phần câu hỏi."
        "Ví dụ: Câu 5 có đáp án đúng là B thì câu hỏi sẽ được hiển thị như sau: Câu 5: Màng sinh chất có chức năng gì? \n A. Bảo vệ tế bào. \n  *B. Chuyển hóa năng lượng. \n C. Tổng hợp DNA \n D. Tạo thành ribosome. \n"
        "Chỉ có đáp án đúng mới có *. Các đáp án còn lại không có *.\n"
        "11. Chuyển tất cả ký tự '*' thành '\*'.\n"
        "12. Đối với các câu hỏi trắc nghiệm, phần câu hỏi, mỗi đáp án ở một hàng và không cùng hàng với câu hỏi."
        "Bạn có thể làm được việc đó bằng cách thêm \n vào đầu mỗi đáp án.\n"
        "13. Tất cả các output bằng tiếng Việt. Không in nghiêng hay bôi đậm."
        "Đưa tất cả các output vào file markdown.\n"
    ),
    expected_output="Một bộ đề kiểm tra và bài tập chi tiết cho chủ đề sub_topics của {topic},"
        "bao gồm câu hỏi trắc nghiệm, đáp án và giải thích. Trả lại output theo dạng Markdown.",
    agent=test_creator,
    output_file="test.md",
    context=[planning_task, lesson_plan_task]
)

In [13]:
crew = Crew(
    agents=[class_planner, lesson_designer, test_creator],
    #agents=[test_creator],
    tasks=[planning_task,lesson_plan_task, test_assignment_task],
    #tasks=[test_assignment_task],
    verbose=2
)


In [14]:
result = crew.kickoff(inputs={'topic': 'Môn vật lý lớp 10',
                              'range': '6 tuần',
                              'students': '30 em học sinh giỏi, 15 em học sinh khá, và 5 em học sinh trung bình'
                              })
print(result)

 [2024-08-16 02:40:34][DEBUG]: == Working Agent: Giáo viên Lập Kế Hoạch Nội Dung Giảng dạy
 [2024-08-16 02:40:34][INFO]: == Starting Task: 1. Tìm trong Milvus các thông tin liên quan đến Appendix (mục lục) bằng cách pass input query='chương và bài học trích từ file mục_lục.pdf cho môn học' và subject = Môn vật lý lớp 10.Dựa vào các thông tin đã tìm được trong Milvus, giới hạn các chủ đề phù hợp trong vòng 6 tuần và phù hợp với các học sinh 30 em học sinh giỏi, 15 em học sinh khá, và 5 em học sinh trung bình.bao gồm các khái niệm cơ bản, định lý, và có thể bao gồm ứng dụng thực tế của chủ đề.Nếu Môn vật lý lớp 10 có dấu thanh tiếng việt, hãy loại bỏ dấu thanh tiếng việt.Lấy đúng nội dung. Không lấy linh tinh. Chỉ tìm trong phạm vi liên quan, không tìm lan man.Xác định các khái niệm và kỹ năng cần kiểm tra cho từng chủ đề.Lưu ý: Khi gặp cảnh báo này 'I tried reusing the same input, I must stop using this action input. I'll try something else instead.', lập tức dừng query, chuyển sang bướ

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


 

data: ['["id: 451393591140440479, distance: 36.653133392333984, entity: {\'page_number\': 1, \'filename\': \'muc_luc_tai_lieu_vat_ly.pdf\', \'text\': \'File title muc_luc_tai_lieu_vat_ly: <= ỤC LỤC\'}", "id: 451393591140440481, distance: 38.69627380371094, entity: {\'page_number\': 1, \'filename\': \'muc_luc_tai_lieu_vat_ly.pdf\', \'text\': \'File title muc_luc_tai_lieu_vat_ly: Bài 20. Một số ví dụ về cách giải các bài toán thuộc phần động lực học Bài 21. Moment lực. Cân bằng của vật rắn Bài 22. Thực hành: Tổng hợp lực Bài 23. Năng lượng. Công cơ học Bài 24. Công suất Bài 25. Động năng, thê năng Bài 26. Cơ năng và định luật bảo toàn cơnăng lành: Xác định động trước và sau va chạm Bài 31. Động học của chuyển động tròn đều _ .Bài 32. Lực hướng tâm và gia tốc hướng tâm Bài 33. Biên dạng của vật rắn Bài 34. Khối lượng riêng. Áp suất chất lỏng Giải thích một số thuật ngữ dùng\'}", "id: 451393591140440480, distance: 39.17973709106445, entity: {\'page_number\': 1, \'filename\': \'muc_luc_t